In [16]:
import os

In [17]:
USER = 'thomas'
MOVIES_CSV = f'movies_for_{USER}_to_rate.csv'
IMGS_DIR = os.path.join('..', 'images', 'posters')

In [18]:
import sys
import custompigeon as pigeon
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import Image, display, TextDisplayObject

In [19]:
sys.path.insert(0, os.path.join(os.path.abspath('..'), 'images'))
from download import download_poster

In [20]:
savedir = f'{USER}_preferences.csv'
if os.path.exists(savedir):
    print(f'Preferences file {savedir!r} already exists (loading rated movies)')
    preloaded = pd.read_csv(savedir)
else:
    preloaded = pd.DataFrame(columns=['imdb_id', 'title'])

Preferences file 'thomas_preferences.csv' already exists (loading rated movies)


In [21]:
def get_img_path_from_id(imdb_id):
    return os.path.join(IMGS_DIR, f'{imdb_id}.jpeg')

In [22]:
def display_fn(row):
    imdb_id, title = row
    return display(title, Image(get_img_path_from_id(imdb_id)))

In [23]:
def annotations_to_df(annot):
    return pd.DataFrame([(imdb_id, title, rating) for (imdb_id, title), rating in annot], 
                        columns=('imdb_id', 'title', 'rating'))

In [24]:
def reduce_already_rated(movies, preloaded):
    return movies[~movies['imdb_id'].isin(preloaded['imdb_id'])]

In [25]:
def download_missing_photos(reduced):
    for imdb_id in tqdm(reduced['imdb_id']):
        download_poster(imdb_id, get_img_path_from_id(imdb_id))

In [26]:
def annotations_to_ratings_df(annot, preloaded):
    return pd.concat([annotations_to_df(annot), preloaded]).drop_duplicates('imdb_id')

In [27]:
movies = pd.read_csv(MOVIES_CSV, usecols=('title', 'imdb_id'))
reduced = reduce_already_rated(movies, preloaded)

In [28]:
download_missing_photos(reduced)

100%|██████████| 3/3 [00:00<00:00, 2984.56it/s]


In [29]:
annot = pigeon.annotate(reduced.loc[:, ['imdb_id', 'title']].values, 
                        options=['1', '2', '3', '4', '5', '?'],
                        include_skip=False,
                        display_fn=display_fn)

HTML(value='0 examples annotated, 4 examples left')

Output()

In [15]:
annotations_to_ratings_df(annot, preloaded).to_csv(savedir, index=False)